#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [6]:
# Loading document 
loader = TextLoader("data_ingestion/speech.txt")
documents = loader.load()
## split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 30)
docs = text_splitter.split_documents(documents)

In [7]:
docs

[Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='…\n\nIt will be all th

In [9]:
embeddings = OllamaEmbeddings(model = "gemma2:2b")
db = FAISS.from_documents(docs, embeddings)

In [10]:
db

In [11]:
query = "How does the speaker describe the desried outcome of the war?"
## this query is based on the text in the speech document 
docs= db.similarity_search(query)
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

In [12]:
## docs here would contain the parts of the document which are in relevant context compared to the query asked

<h3> as a retriever </h3>
we can convert vector store into the retriever class this allows us to use it in other langchain  methods which largely work with retrievers

In [13]:
retriever = db.as_retriever()
retriever.invoke(query)

[Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when Amer

In [ ]:
## this gives same reults this is useful when we want to use multiple models 
## in taht case we would be converting the database into retirever and then using queries on that

In [14]:
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [16]:
docs_and_Score = db.similarity_search_with_score(query)

In [17]:
docs_and_Score

[(Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  7445.2163),
 (Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day ha

In [18]:
##  we can also pass vectors instead of sentences
embeddings_vector = embeddings.embed_query(query)
embeddings_vector

[1.571416974067688,
 -0.1776846945285797,
 -0.6355425715446472,
 -1.5673736333847046,
 -0.8732309937477112,
 -0.4436841309070587,
 -0.10870799422264099,
 1.6702874898910522,
 -0.9138590693473816,
 -0.13403846323490143,
 -0.07886045426130295,
 -0.39490044116973877,
 -1.1600961685180664,
 -0.6399040818214417,
 -2.7618091106414795,
 -1.9951568841934204,
 0.829016387462616,
 -0.9203038811683655,
 -2.0734364986419678,
 -1.2348012924194336,
 0.6163944005966187,
 -1.8004631996154785,
 -2.9253156185150146,
 1.6760516166687012,
 -1.0682611465454102,
 -1.7173428535461426,
 0.7219064831733704,
 -0.9777401089668274,
 -1.4945616722106934,
 2.140416383743286,
 -2.4426791667938232,
 -0.8945929408073425,
 0.40124839544296265,
 1.102112889289856,
 1.415124535560608,
 1.0506291389465332,
 -0.38294747471809387,
 1.3216700553894043,
 2.3966808319091797,
 -2.992765426635742,
 1.6165738105773926,
 0.27152717113494873,
 -0.6718770265579224,
 -2.438720226287842,
 -3.9729983806610107,
 -1.6934517621994019,
 0.

In [19]:
db.similarity_search_by_vector(embeddings_vector)

[Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='To such a task we can dedicate our lives and our fortunes, everything that we are and everything that we have, with the pride of those who know that the day has come when Amer

In [20]:
## saving and loading of vector store 

 # db has all the embeddings of the vectors based on the ollama model we used

db.save_local("faiss_index")

In [22]:
new_db = FAISS.load_local("faiss_index",embeddings, allow_dangerous_deserialization=True)
##while loading we also need to tell the model which embeddings we are going to use 

In [23]:
docs = new_db.similarity_search(query)

In [24]:
docs[0]

Document(metadata={'source': 'data_ingestion/speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.')